In [1]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
from bs4  import BeautifulSoup
import json 
from tqdm import tqdm
import time
from dotenv import load_dotenv
import ast
import os

In [2]:
load_dotenv()

True

In [3]:
YOUR_API_KEY = os.getenv('API_KEY')

In [5]:
df = pd.read_csv('precio_mes_combustible.csv')

In [ ]:
df.head(2)

<h1><center>Geocoding Api's</center></h1>

## Google API

* [**Google_Free_Tier**](https://cloud.google.com/free/?utm_source=google&utm_medium=cpc&utm_campaign=latam-LATAM-all-es-dr-AKWS-all-all-trial-p-dr-1011454-LUAC0013610&utm_content=text-ad-none-any-DEV_c-CRE_540885993580-ADGP_Hybrid%20%7C%20BKWS%20-%20PHR%20%7C%20Txt%20~%20Management-Tools_Console-KWID_43700065922071537-kwd-651816590614&utm_term=KW_google%20console-ST_Google%20Console&gclid=Cj0KCQjwnbmaBhD-ARIsAGTPcfWVMlJlTFywFNaAo8P-MNL4_gM6L7yucqAQ37xg1AIEHzpAzF_kkAoaAsszEALw_wcB&gclsrc=aw.ds)
* [**Google_Api_Pricing**](https://mapsplatform.google.com/pricing/)

## HERE Platform

* [**Here_Register**](https://platform.here.com/sign-up?step=verify-identity&aid=www.here.com-start-building)
* [**Here_Princing**](https://www.here.com/pricing)
* [**Geocoding_Documentation**](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-geocode-brief.html)

# Clean Data 

In [7]:
df[df['Periodo']==2022]['Mes'].max()

2

In [8]:
df2 =  df[(df['Periodo']==2022) & (df['Mes']==2)]
df2 =  df2[(df2['Departamento']=='ANTIOQUIA') | (df2['Departamento']=='BOGOTA D.C.')]

In [27]:
df2['Municipio'].value_counts().head(10)

BOGOTA  D.C.    1368
Name: Municipio, dtype: int64

In [28]:
df3 = df2[(df2['Departamento']=='BOGOTA D.C.') | ((df2['Departamento']=='ANTIOQUIA') & ((df2['Municipio']=='MEDELLIN') | (df2['Municipio']=='ITAGUI') | (df2['Municipio']=='BELLO') | (df2['Municipio']=='SABANETA') | (df2['Municipio']=='RIONEGRO') | (df2['Municipio']=='ENVIGADO')))]
df3.reset_index(inplace = True)
df3.drop(columns = 'index',inplace = True)

In [ ]:
df3.head(2)

In [ ]:
df3['Municipio'].unique()

In [37]:
df3['Dir2'] = df3['Direccion'].apply(lambda x : x.replace('#','No'))
df3['Full_Address'] = df3.apply(lambda x : x['Direccion'] + ', ' + x['Municipio'].capitalize() + ', Colombia' , axis = 1)

In [ ]:
df3['Full_Address'].nunique()

In [ ]:
df3['Full_Address'][0]

In [40]:
estaciones =  df3[['Nombre_comercial','Direccion','Full_Address']].drop_duplicates(subset = 'Nombre_comercial')
estaciones.reset_index(inplace = True)
estaciones.drop(columns = 'index',inplace = True)

In [ ]:
estaciones['Full_Address'][0]

In [ ]:
address = estaciones['Full_Address'][0]
url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
response = requests.get(url).json()

In [48]:
def Get_Coords(address,YOUR_API_KEY):
    
    url = f'https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={YOUR_API_KEY}'
    
    try : 
        
        response = requests.get(url).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LNG = response['items'][0]['position']['lng']
        
        results = [CleanAddress,LAT,LNG]
        
    except :
        
        results = ['Not Found','NA','NA']
        
    
    return results 

In [ ]:
Get_Coords(address,YOUR_API_KEY)

# Extracción de coordenadas 

In [50]:
coordenadas = []
tiempo = [1,2,2.5]

for i in tqdm(range(len(estaciones['Full_Address'])),colour = 'green'):
    
    try :
        
        R = Get_Coords(estaciones['Full_Address'][i],YOUR_API_KEY)
        geo_source = R[1],R[2]
        coordenadas.append(geo_source)
        
    except :
        
        coordenadas.append('Error')
        
    
    time.sleep(random.choices(tiempo)[0])
    
        

100%|██████████| 376/376 [14:37<00:00,  2.33s/it]


In [ ]:
coordenadas

In [52]:
estaciones['Coords'] = coordenadas

In [ ]:
len(df3)

In [ ]:
estaciones.head(2)

In [55]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [56]:
df_final.head(3)

,Periodo,Mes,Codigo_departamento,Departamento,Codigo_municipio,Municipio,Nombre_comercial,Bandera,Direccion,Producto,Precio,Estado,Dir2,Full_Address,Coords
0,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA CORRIENTE OXIGENADA,9860,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
1,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,BIODIESEL EXTRA,9780,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"
2,2022,2,11,BOGOTA D.C.,11001,BOGOTA D.C.,SUPER ESTACION DE SERVICIO TEXACO 10,TEXACO,AVENIDA CARRERA 7 No. 123A-48,GASOLINA EXTRA OXIGENADA,17250,A,AVENIDA CARRERA 7 No. 123A-48,"AVENIDA CARRERA 7 No. 123A-48, Bogota d.c., C...","(4.69921, -74.02878)"


In [57]:
df_final = pd.merge(df3,estaciones[['Full_Address','Coords']],how = 'inner' , on = 'Full_Address')

In [58]:
df_final.to_csv('DATASET_FINAL.CSV',index = False)